In [2]:
# Classification model code
import pandas as pd
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import LabelEncoder
#from sklearn.metrics import classification_report, accuracy_score

In [1]:
!pip install scikit-learn==1.5.1

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 21.8 MB/s eta 0:00:0000:010:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2


In [3]:
df = pd.read_parquet('model_data_1.parquet')

In [4]:

df.head()


,locationid,ridership,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2.0,1.0,2022-02-06 15:00:00,-5.6,0.0,0.0,11.0,0.0,0.0,35.0,6.600000,331.0
1,2.0,1.0,2022-02-16 23:00:00,5.4,0.0,0.0,29.0,0.0,0.0,97.0,12.100000,280.0
2,2.0,1.0,2022-02-23 08:00:00,13.5,0.0,0.0,44.0,8.0,11.0,100.0,16.299999,227.0
3,2.0,4.0,2022-02-23 19:00:00,17.5,0.0,0.0,22.0,5.0,2.0,54.0,21.600000,184.0
4,2.0,1.0,2022-03-22 10:00:00,7.4,0.0,0.0,20.0,0.0,30.0,8.0,10.800000,127.0


In [5]:

categorical_columns = df[['locationid']].columns

# Convert data type to int for these columns - gets rid of decimal place
for column in categorical_columns:
    df[column] = df[column].astype('int')
    

integer_columns = df[['ridership']].columns

# Convert data type to object for these columns
for column in integer_columns:
    df[column] = df[column].astype('int')
    

In [6]:

categorical_columns = df[['locationid']].columns

# Convert data type to category for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')
    

In [7]:

# Check the data types in each column as assigned by default.
df.dtypes


locationid                     category
ridership                         int64
time                     datetime64[ns]
temperature_2m (°C)             float32
precipitation (mm)              float32
rain (mm)                       float32
cloudcover (%)                  float32
cloudcover_low (%)              float32
cloudcover_mid (%)              float32
cloudcover_high (%)             float32
windspeed_10m (km/h)            float32
winddirection_10m (°)           float32
dtype: object

In [8]:

# Check just how many rows:
df.shape


(2143060, 12)

In [9]:

df = df.drop(columns=['temperature_2m (°C)', 'precipitation (mm)', 'rain (mm)','winddirection_10m (°)', 'windspeed_10m (km/h)', 'cloudcover_high (%)',
                      'cloudcover_mid (%)', 'cloudcover_low (%)', 'cloudcover (%)'])

df.head()


,locationid,ridership,time
0,2,1,2022-02-06 15:00:00
1,2,1,2022-02-16 23:00:00
2,2,1,2022-02-23 08:00:00
3,2,4,2022-02-23 19:00:00
4,2,1,2022-03-22 10:00:00


In [10]:

df.isnull().sum()


locationid         0
ridership          0
time          634050
dtype: int64

In [11]:

df=df.dropna()


In [12]:

df.isnull().sum()


locationid    0
ridership     0
time          0
dtype: int64

In [13]:

df.shape


(1509010, 3)

In [14]:

# Filter the dataframe to keep only the rows between 6pm to 6am
df = df[(df['time'].dt.hour < 6) | (df['time'].dt.hour >= 18)]


In [16]:
# Filter the data to only keep data for July and August
df = df[(df['time'].dt.month == 7) | (df['time'].dt.month == 8)]

In [19]:
df.shape


(147194, 3)

In [20]:
df.head(10)

,locationid,ridership,time
22,2,1,2022-08-16 00:00:00
23,2,1,2022-08-17 23:00:00
26,2,1,2022-08-24 22:00:00
28,2,2,2022-08-29 21:00:00
3646,3,17,2022-07-01 00:00:00
3647,3,4,2022-07-01 01:00:00
3648,3,4,2022-07-01 02:00:00
3649,3,14,2022-07-01 03:00:00
3650,3,42,2022-07-01 04:00:00
3651,3,365,2022-07-01 05:00:00


In [21]:

df.to_parquet('night_data.parquet')


In [22]:

df = pd.read_parquet('night_data.parquet')


In [23]:

df.dtypes


locationid             int64
ridership              int64
time          datetime64[ns]
dtype: object

In [24]:

# Find the minimum and maximum ridership values
print("Minimum ridership: ", df['ridership'].min())
print("Maximum ridership: ", df['ridership'].max())

max_ridership = df['ridership'].max()


Minimum ridership:  0
Maximum ridership:  41070


In [25]:

# Count the number of rows in each specified range
#ranges = {
    #'Quiet': (0, 999),
    #'Not Too Busy': (1000, 4999),
    #'A Little Busy': (5000, 9999),
    #'Busy': (10000, 19999),
    #'Very Busy': (20000, 49999),
    #'Extremely Busy': (50000, 74080)
    #}

# Define ranges using quantiles to get more balanced distribution
quantiles = df['ridership'].quantile([0, 0.2, 0.4, 0.6, 0.8, 0.99]).values

# Max ridership is an outlier and without the following code was the only value in extremely busy
quantiles = list(quantiles) + [max_ridership]
    
# Convert quantile values to integers to get rid of decimal places
quantiles = [int(q) for q in quantiles]

# Define ranges based on quantiles
# minus 1 at the end of each quantile to avoid overlap, e.g. Quiet was 0-23 and not too busy was 23-143, meaning both contained 23
ranges = {
    'Quiet': (quantiles[0], quantiles[1]-1),
    'Not Too Busy': (quantiles[1], quantiles[2]-1),
    'A Little Busy': (quantiles[2], quantiles[3]-1),
    'Busy': (quantiles[3], quantiles[4]-1),
    'Very Busy': (quantiles[4], quantiles[5]-1),
    'Extremely Busy': (quantiles[5], quantiles[6])
    }
    
range_counts = {}
for label, (lower, upper) in ranges.items():
    count = ((df['ridership'] >= lower) & (df['ridership'] <= upper)).sum()
    range_counts[label] = count
    
print("Number of entries in each ridership range:")
for label, count in range_counts.items():
    print(f"{label}: {count}")
    
print("\n")    
print("Ranges used for each category:")
for label, (lower, upper) in ranges.items():
    print(f"{label}: {lower} - {upper}")
    

Number of entries in each ridership range:
Quiet: 28879
Not Too Busy: 29797
A Little Busy: 29539
Busy: 29528
Very Busy: 27978
Extremely Busy: 1473


Ranges used for each category:
Quiet: 0 - 9
Not Too Busy: 10 - 52
A Little Busy: 53 - 154
Busy: 155 - 522
Very Busy: 523 - 8449
Extremely Busy: 8450 - 41070


In [26]:

# Assign busyness label based on ridership value
def assign_label(ridership):
    if 0 <= ridership <= 9:
        return 'Quiet'
    elif 10 <= ridership <= 52:
        return 'Not Too Busy'
    elif 53 <= ridership <= 154:
        return 'A Little Busy'
    elif 155 <= ridership <= 522:
        return 'Busy'
    elif 523 <= ridership <= 8449:
        return 'Very Busy'
    elif 8450 <= ridership <= 41070:
        return 'Extremely Busy'
    else:
        raise ValueError(f"Ridership value {ridership} cannot be assigned a category.")

# Create new column 'busyness' assigning label to each ridership value
try:
    df['busyness'] = df['ridership'].apply(assign_label)
    print("Label assignment successful.")
except ValueError as e:
    print(f"Error: {str(e)}")

# Display the updated DataFrame with the new column
print(df.tail(30))


Label assignment successful.
         locationid  ridership                time       busyness
2136455         263       1229 2022-08-29 18:00:00      Very Busy
2136456         263        790 2022-08-29 19:00:00      Very Busy
2136457         263        616 2022-08-29 20:00:00      Very Busy
2136458         263        534 2022-08-29 21:00:00      Very Busy
2136459         263        447 2022-08-29 22:00:00           Busy
2136460         263        281 2022-08-29 23:00:00           Busy
2136461         263        126 2022-08-30 00:00:00  A Little Busy
2136462         263         54 2022-08-30 01:00:00  A Little Busy
2136463         263         34 2022-08-30 02:00:00   Not Too Busy
2136464         263         16 2022-08-30 03:00:00   Not Too Busy
2136465         263         20 2022-08-30 04:00:00   Not Too Busy
2136466         263         77 2022-08-30 05:00:00  A Little Busy
2136479         263       1492 2022-08-30 18:00:00      Very Busy
2136480         263       1041 2022-08-30 19:00

In [27]:

df.to_parquet('night_data1.parquet')


In [28]:

df = pd.read_parquet('night_data1.parquet')

df.head(10)


,locationid,ridership,time,busyness
22,2,1,2022-08-16 00:00:00,Quiet
23,2,1,2022-08-17 23:00:00,Quiet
26,2,1,2022-08-24 22:00:00,Quiet
28,2,2,2022-08-29 21:00:00,Quiet
3646,3,17,2022-07-01 00:00:00,Not Too Busy
3647,3,4,2022-07-01 01:00:00,Quiet
3648,3,4,2022-07-01 02:00:00,Quiet
3649,3,14,2022-07-01 03:00:00,Not Too Busy
3650,3,42,2022-07-01 04:00:00,Not Too Busy
3651,3,365,2022-07-01 05:00:00,Busy


In [29]:
df.shape

(147194, 4)

In [30]:

df.dtypes


locationid             int64
ridership              int64
time          datetime64[ns]
busyness              object
dtype: object

In [31]:

categorical_columns = df[['locationid', 'busyness']].columns

# Convert data type to category for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')
    

In [32]:

df.dtypes



locationid          category
ridership              int64
time          datetime64[ns]
busyness            category
dtype: object

# Models

- First need to extract the data needed from datetime, i.e. day, month, year, hour.
- Then encode busyness column as numeric

In [33]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [34]:

# Extract features from datetime
df['hour'] = df['time'].dt.hour
df['day'] = df['time'].dt.day
df['month'] = df['time'].dt.month
df['weekday'] = df['time'].dt.weekday


In [35]:

df.head()


,locationid,ridership,time,busyness,hour,day,month,weekday
22,2,1,2022-08-16 00:00:00,Quiet,0,16,8,1
23,2,1,2022-08-17 23:00:00,Quiet,23,17,8,2
26,2,1,2022-08-24 22:00:00,Quiet,22,24,8,2
28,2,2,2022-08-29 21:00:00,Quiet,21,29,8,0
3646,3,17,2022-07-01 00:00:00,Not Too Busy,0,1,7,4


In [36]:

# Drop time column and ridership
df = df.drop(columns=['ridership', 'time'])


In [37]:

# Encode busyness
le = LabelEncoder()
df['busyness'] = le.fit_transform(df['busyness'])


In [38]:

# Target feature
y = df['busyness']
y.head()


22      4
23      4
26      4
28      4
3646    3
Name: busyness, dtype: int64

In [39]:

# Set all remaining features as X, remove y from this subset
X = df.drop(columns=['busyness'])
X.head()


,locationid,hour,day,month,weekday
22,2,0,16,8,1
23,2,23,17,8,2
26,2,22,24,8,2
28,2,21,29,8,0
3646,3,0,1,7,4


In [40]:

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)


In [41]:

# Initialize and train the model
model = RandomForestClassifier(random_state=2)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=2)

In [42]:

# Make predictions
y_pred = model.predict(X_test)


In [43]:

from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Example: Classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.42806675875812406
              precision    recall  f1-score   support

           0       0.36      0.35      0.35      8953
           1       0.41      0.41      0.41      8816
           2       0.42      0.37      0.40       466
           3       0.44      0.45      0.45      8812
           4       0.41      0.39      0.40      8724
           5       0.52      0.55      0.53      8388

    accuracy                           0.43     44159
   macro avg       0.43      0.42      0.42     44159
weighted avg       0.43      0.43      0.43     44159



In [44]:

from datetime import datetime, timedelta
import pytz


In [45]:

# New York timezone
ny_tz = pytz.timezone('America/New_York')


In [46]:

# Define a mapping dictionary for busyness labels
label_mapping = {
    0: 'Quiet',
    1: 'Not Busy',
    2: 'A Little Busy',
    3: 'Busy',
    4: 'Very Busy',
    5: 'Extremely Busy'
}

In [47]:

def predict_busyness(locationid, model):
    # Current datetime - New York
    now = datetime.now(ny_tz)
    
    # Datetime for 1 hour from now
    future_time = now + timedelta(hours=1)
    
    # Extract hour, day of week, and month - values model is trained on
    hour = future_time.hour
    day = future_time.day
    month = future_time.month
    weekday = future_time.weekday()
    
    # Input features
    input_data = pd.DataFrame({
        'locationid': [locationid],
        'hour': [hour],
        'day': [day],
        'month': [month],
        'weekday': [weekday]
    })
    
    input_data['locationid'] = input_data['locationid'].astype('category')
    input_data['day'] = input_data['day'].astype('category')
    input_data['month'] = input_data['month'].astype('category')
    input_data['weekday'] = input_data['weekday'].astype('category')
    
    
    prediction = model.predict(input_data)
    
    # numeric prediction to its corresponding busyness label 
    predicted_label = label_mapping[prediction[0]]
    
    return predicted_label



In [56]:

# Input data - sample
location_id = 6

predicted_busyness = predict_busyness(location_id, model)
print(f"Predicted busyness at location {location_id} 1 hour from now in New York: {predicted_busyness}")


Predicted busyness at location 6 1 hour from now in New York: Extremely Busy


In [57]:

# Import pickle module
import pickle

# Write to file
with open('RF_m1.pkl', 'wb') as file:
    pickle.dump(model, file)